In [1]:
#my_new_env_clone2

%load_ext autoreload
%autoreload 2

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=DeprecationWarning)

import os, sys
#re, gc, joblib
import numpy as np
#import numpy_groupies as npg
import pandas as pd
import scipy.sparse as sp

import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.colors import ListedColormap

import seaborn as sn

import anndata
import scanpy as sc
#import sctk as sk


rcParams["pdf.fonttype"] = 42
rcParams["ps.fonttype"] = 42
np.set_printoptions(linewidth=180)
sc.settings.verbosity = 1
#sk.set_figsize((4, 4))
#expr_cmap = sk.expression_colormap()

In [2]:
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()

2023-05-24 21:33:29.511278: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/R/lib:/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/amd64/server
2023-05-24 21:33:29.511341: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


scanpy==1.8.1 anndata==0.7.6 umap==0.5.1 numpy==1.21.1 scipy==1.8.1 pandas==1.3.1 scikit-learn==0.24.2 statsmodels==0.13.2 python-igraph==0.9.6 pynndescent==0.5.4


In [3]:
root_path = '/lustre/scratch126/cellgen/team298/hg6/Fetal_skin/data/final/'
save_path = '/home/jovyan/projects/skin/fetal_skin/'
sc.settings.figdir = save_path + 'figures/other plots/'

# Load and configure data object

In [4]:
#load pooled prenatal skin, healthy adult and skin organoid object

pooled_object = 'pooled_fetal_organoid_adult.processed.h5ad'
pooled_path = root_path + pooled_object
pdata = sc.read(pooled_path)
pdata

AnnData object with n_obs × n_vars = 506107 × 46794
    obs: 'sanger_id', 'chemistry', 'batch', 'gender', 'pcw', 'sorting', 'sample', 'chemistry_sorting', 'cell_caller', 'scrublet_score', 'cluster_scrublet_score', 'doublet_pval', 'bh_doublet_pval', 'n_counts', 'n_genes', 'percent_mito', 'percent_ribo', 'percent_hb', 'independent_annotation_refined', 'independent_annotation_broad1', 'independent_annotation_broad2', 'independent_annotation_broad3', 'independent_annotation_broad4', 'independent_annotation_broad5', 'joint_annotation', 'is_maternal', 'fig1b_annotation', 'independent_annotation_refined_20220201a', 'independent_annotation_refined_20220201b', 'independent_annotation_broad1_20220201', 'independent_annotation_broad2_20220201', 'independent_annotation_broad3_20220201', 'joint_annotation_20220201a', 'joint_annotation_20220201b', 'fig1b_annotation_20220201', 'day', 'sample_id', 'version', 'week', 'nh3_annot1', 'log1p_n_counts', 'log1p_n_genes', 'n_counts_mito', 'n_counts_ribo', 'n_

In [5]:
print(list(pdata.obs['dataset'].unique()))

['fetal', 'organoid', 'adult']


In [6]:
#slice to prenetal/fetal skin only and remove genes with counts<1
keep1 = ['fetal']
pdata_fetal = pdata[pdata.obs['dataset'].isin(keep1)]
sc.pp.filter_genes(pdata_fetal, min_counts=1, min_cells=None, inplace=True)
pdata_fetal

filtered out 18015 genes that are detected in less than 1 counts


Trying to set attribute `.var` of view, copying.


AnnData object with n_obs × n_vars = 186533 × 28779
    obs: 'sanger_id', 'chemistry', 'batch', 'gender', 'pcw', 'sorting', 'sample', 'chemistry_sorting', 'cell_caller', 'scrublet_score', 'cluster_scrublet_score', 'doublet_pval', 'bh_doublet_pval', 'n_counts', 'n_genes', 'percent_mito', 'percent_ribo', 'percent_hb', 'independent_annotation_refined', 'independent_annotation_broad1', 'independent_annotation_broad2', 'independent_annotation_broad3', 'independent_annotation_broad4', 'independent_annotation_broad5', 'joint_annotation', 'is_maternal', 'fig1b_annotation', 'independent_annotation_refined_20220201a', 'independent_annotation_refined_20220201b', 'independent_annotation_broad1_20220201', 'independent_annotation_broad2_20220201', 'independent_annotation_broad3_20220201', 'joint_annotation_20220201a', 'joint_annotation_20220201b', 'fig1b_annotation_20220201', 'day', 'sample_id', 'version', 'week', 'nh3_annot1', 'log1p_n_counts', 'log1p_n_genes', 'n_counts_mito', 'n_counts_ribo', 'n_

In [7]:
#slice to organoid skin only and check 'day'
keep2 = ['organoid']
pdata_org = pdata[pdata.obs['dataset'].isin(keep2)]
print(list(pdata_org.obs['day'].unique()))

['day-133', 'day-29', 'day-48', 'day-85', 'day-13']


In [8]:
#slice skin organoid object to remove day-13 only and and remove genes with counts<1
pdata_org = pdata_org[pdata_org.obs['day']!='day-13']
print(list(pdata_org.obs['day'].unique()))
sc.pp.filter_genes(pdata_org, min_counts=1, min_cells=None, inplace=True)
pdata_org

['day-133', 'day-29', 'day-48', 'day-85']
filtered out 18681 genes that are detected in less than 1 counts


Trying to set attribute `.var` of view, copying.


AnnData object with n_obs × n_vars = 114278 × 28113
    obs: 'sanger_id', 'chemistry', 'batch', 'gender', 'pcw', 'sorting', 'sample', 'chemistry_sorting', 'cell_caller', 'scrublet_score', 'cluster_scrublet_score', 'doublet_pval', 'bh_doublet_pval', 'n_counts', 'n_genes', 'percent_mito', 'percent_ribo', 'percent_hb', 'independent_annotation_refined', 'independent_annotation_broad1', 'independent_annotation_broad2', 'independent_annotation_broad3', 'independent_annotation_broad4', 'independent_annotation_broad5', 'joint_annotation', 'is_maternal', 'fig1b_annotation', 'independent_annotation_refined_20220201a', 'independent_annotation_refined_20220201b', 'independent_annotation_broad1_20220201', 'independent_annotation_broad2_20220201', 'independent_annotation_broad3_20220201', 'joint_annotation_20220201a', 'joint_annotation_20220201b', 'fig1b_annotation_20220201', 'day', 'sample_id', 'version', 'week', 'nh3_annot1', 'log1p_n_counts', 'log1p_n_genes', 'n_counts_mito', 'n_counts_ribo', 'n_

In [9]:
#check data counts
pdata_fetal.X.data

array([2.2455943, 0.989481 , 0.989481 , ..., 1.6292564, 1.6292564, 1.6292564], dtype=float32)

In [10]:
pdata_fetal.X.max()

11.004526

In [11]:
pdata_org.X.data

array([0.6374563, 0.6374563, 0.6374563, ..., 1.36667  , 1.36667  , 1.36667  ], dtype=float32)

In [12]:
pdata_org.X.max()

8.528311

In [13]:
#re concatenate data to have prenatal skin/organoid object keeping only intersect of genes
import anndata as ad

pdata2 = ad.concat([pdata_fetal, pdata_org])
pdata2

AnnData object with n_obs × n_vars = 300811 × 19372
    obs: 'sanger_id', 'chemistry', 'batch', 'gender', 'pcw', 'sorting', 'sample', 'chemistry_sorting', 'cell_caller', 'scrublet_score', 'cluster_scrublet_score', 'doublet_pval', 'bh_doublet_pval', 'n_counts', 'n_genes', 'percent_mito', 'percent_ribo', 'percent_hb', 'independent_annotation_refined', 'independent_annotation_broad1', 'independent_annotation_broad2', 'independent_annotation_broad3', 'independent_annotation_broad4', 'independent_annotation_broad5', 'joint_annotation', 'is_maternal', 'fig1b_annotation', 'independent_annotation_refined_20220201a', 'independent_annotation_refined_20220201b', 'independent_annotation_broad1_20220201', 'independent_annotation_broad2_20220201', 'independent_annotation_broad3_20220201', 'joint_annotation_20220201a', 'joint_annotation_20220201b', 'fig1b_annotation_20220201', 'day', 'sample_id', 'version', 'week', 'nh3_annot1', 'log1p_n_counts', 'log1p_n_genes', 'n_counts_mito', 'n_counts_ribo', 'n_

In [14]:
pdata2.obs

,sanger_id,chemistry,batch,gender,pcw,sorting,sample,chemistry_sorting,cell_caller,scrublet_score,...,final,dataset,log2pcw,annot_early_late,annot_early_late2,joint_annotation_20220202,fig1b_annotation_v2,independent_annotation_refined_20220202,annot,org_annot0
AAACCTGGTCAGTGGA-1-4834STDY7002879-fetal,4834STDY7002879,SC3Pv2,F16,male,8,CD45P,F16_male_8+1PCW,SC3Pv2_CD45P,Both,0.225806,...,nan,fetal,3.000000,fetal_Macrophage,fetal_Macrophage,LYVE1++ macrophage,Macrophage,LYVE1++ macrophage,fetal_Macrophage,nan
AAAGATGGTCGATTGT-1-4834STDY7002879-fetal,4834STDY7002879,SC3Pv2,F16,male,8,CD45P,F16_male_8+1PCW,SC3Pv2_CD45P,Both,0.149606,...,nan,fetal,3.000000,fetal_Monocyte,fetal_Monocyte,Monocyte (activated/differentiating),Monocyte,Monocyte (activated/differentiating),fetal_Monocyte,nan
AAAGCAAAGATGTGGC-1-4834STDY7002879-fetal,4834STDY7002879,SC3Pv2,F16,male,8,CD45P,F16_male_8+1PCW,SC3Pv2_CD45P,Both,0.201970,...,nan,fetal,3.000000,fetal_Macrophage,fetal_Macrophage,LYVE1++ macrophage,Macrophage,LYVE1++ macrophage,fetal_Macrophage,nan
AAAGTAGCAGATCGGA-1-4834STDY7002879-fetal,4834STDY7002879,SC3Pv2,F16,male,8,CD45P,F16_male_8+1PCW,SC3Pv2_CD45P,Both,0.164557,...,nan,fetal,3.000000,fetal_Progenitor,fetal_Progenitor,Eo/baso/mast cell progenitor,Haem progenitor,Eo/baso/mast cell progenitor,fetal_Haem progenitor,nan
AAAGTAGTCCGCATCT-1-4834STDY7002879-fetal,4834STDY7002879,SC3Pv2,F16,male,8,CD45P,F16_male_8+1PCW,SC3Pv2_CD45P,Both,0.181818,...,nan,fetal,3.000000,fetal_Macrophage,fetal_Macrophage,LYVE1++ macrophage,Macrophage,LYVE1++ macrophage,fetal_Macrophage,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCACAGCGAACA-1-11-organoid,nan,nan,WA25,nan,12,nan,nan,nan,nan,0.062500,...,nan,organoid,3.584963,organoid_Fibroblast,organoid_Fibroblast,nan,nan,nan,organoid_Fibroblast,organoid_Pre-dermal condensate
TTTGTCACATGACATC-1-11-organoid,nan,nan,WA25,nan,12,nan,nan,nan,nan,0.073298,...,nan,organoid,3.584963,organoid_Fibroblast,organoid_Fibroblast,nan,nan,nan,organoid_Fibroblast,organoid_Fibroblasts POSTN+
TTTGTCACATGCCCGA-1-11-organoid,nan,nan,WA25,nan,12,nan,nan,nan,nan,0.111111,...,nan,organoid,3.584963,organoid_Fibroblast,organoid_Fibroblast,nan,nan,nan,organoid_Fibroblast,organoid_Fibroblasts POSTN+
TTTGTCATCAGGCAAG-1-11-organoid,nan,nan,WA25,nan,12,nan,nan,nan,nan,0.043069,...,nan,organoid,3.584963,organoid_Fibroblast,organoid_Fibroblast,nan,nan,nan,organoid_Fibroblast,organoid_Pre-dermal condensate


In [15]:
#drop unnecessary metadata columns
pdata2.obs.drop(columns=['cell_caller', 'scrublet_score', 'cluster_scrublet_score', 'doublet_pval', 'bh_doublet_pval', 
                         'n_counts', 'n_genes', 'percent_mito', 'percent_ribo', 'percent_hb',
                         'independent_annotation_refined', 'independent_annotation_broad1', 'independent_annotation_broad2', 
                         'independent_annotation_broad3', 'independent_annotation_broad4', 'independent_annotation_broad5', 
                         'joint_annotation', 'is_maternal', 'fig1b_annotation', 'independent_annotation_refined_20220201a', 
                         'independent_annotation_refined_20220201b', 'independent_annotation_broad1_20220201', 
                         'independent_annotation_broad2_20220201', 'independent_annotation_broad3_20220201', 
                         'joint_annotation_20220201a', 'joint_annotation_20220201b', 'fig1b_annotation_20220201',
                         'nh3_annot1', 'log1p_n_counts', 'log1p_n_genes', 'n_counts_mito', 'n_counts_ribo', 'n_counts_hb', 
                         'percent_top50', 'pass_auto_filter', 'scrublet_score_z', 'bh_pval', 'scrublet_done', 'doublet', 'seurat_clusters',
                        'fsk_annot1_min30', 'fsk_annot1_min30_prob', 'fsk_annot2_min30', 'fsk_annot2_min30_prob', 'day8to13', 
                         'org_annot0_prob',  'org_annot30_prob', 'fsk_annot0', 'fsk_annot0_prob', 'fsk_annot30', 'fsk_annot30_prob',
                         'mad_prd', 'Status', 'Site', 'Site_old', 'Tissue', 'Enrichment', 'Location', 'Sex', 'Age', 'stage',
                         'anno_final', 'final','log2pcw', 'annot_early_late', 'annot_early_late2',
                        ], inplace=True)

In [16]:
pdata2

AnnData object with n_obs × n_vars = 300811 × 19372
    obs: 'sanger_id', 'chemistry', 'batch', 'gender', 'pcw', 'sorting', 'sample', 'chemistry_sorting', 'day', 'sample_id', 'version', 'week', 'org_annot30', 'dataset', 'joint_annotation_20220202', 'fig1b_annotation_v2', 'independent_annotation_refined_20220202', 'annot', 'org_annot0'
    obsm: 'X_pca_project', 'X_umap'

In [17]:
# check annotations
print(list(pdata2.obs['joint_annotation_20220202'].unique()))
print(list(pdata2.obs['org_annot0'].unique()))

['LYVE1++ macrophage', 'Monocyte (activated/differentiating)', 'Eo/baso/mast cell progenitor', 'DC2', 'TREM2+ macrophage', 'Monocyte precursor', 'Monocyte', 'WNT2+ fibroblast', 'Iron-recycling macrophage', 'NK cell', 'B cell', 'LTi', 'ILC3', 'Inflammatory DC', 'MEMP - Early erythroid', 'CD4 T cell', 'Pro B cell', 'HSC', 'Pre B cell', 'MHCII+ macrophage', 'LE', 'Pre pro B cell', 'Innate T type1', 'CD8 T cell', 'Myocytes', 'pDC', 'Mast cell (most mature)', 'DC1', 'Mast cell (medium)', 'Capillaries', 'Neutrophil2', 'Capillary arterioles', 'LC', 'Megakaryocyte', 'MEMP - Megak', 'Mast cell (earliest)', 'Erythroid (fetal)', 'Myoblasts', 'Myofibroblasts', 'Pre-dermal condensate', 'HOXC5+ early fibroblast', 'PEAR1+ fibroblast', 'Pericytes', 'FRZB+ early fibroblast', 'Early erythroid (embryonic)', 'Early myocytes', 'Neuron progenitors', 'Myelinating Schwann cells', 'Schwann/Schwann precursors', 'LMCD1+ mural cell', 'PLN+ mural cell', 'ILC2', 'Lymphoid progenitor', 'SPP1+ proliferating neuron pr

In [18]:
#create combined annotation column
pdata2.obs['pooled_annot'] = pdata2.obs['org_annot0']
cond = (pdata2.obs['pooled_annot'] == 'nan')
pdata2.obs.loc[cond, 'pooled_annot'] = 'fetal_' + pdata2.obs['joint_annotation_20220202']
print(list(pdata2.obs['pooled_annot'].unique()))

['fetal_LYVE1++ macrophage', 'fetal_Monocyte (activated/differentiating)', 'fetal_Eo/baso/mast cell progenitor', 'fetal_DC2', 'fetal_TREM2+ macrophage', 'fetal_Monocyte precursor', 'fetal_Monocyte', 'fetal_WNT2+ fibroblast', 'fetal_Iron-recycling macrophage', 'fetal_NK cell', 'fetal_B cell', 'fetal_LTi', 'fetal_ILC3', 'fetal_Inflammatory DC', 'fetal_MEMP - Early erythroid', 'fetal_CD4 T cell', 'fetal_Pro B cell', 'fetal_HSC', 'fetal_Pre B cell', 'fetal_MHCII+ macrophage', 'fetal_LE', 'fetal_Pre pro B cell', 'fetal_Innate T type1', 'fetal_CD8 T cell', 'fetal_Myocytes', 'fetal_pDC', 'fetal_Mast cell (most mature)', 'fetal_DC1', 'fetal_Mast cell (medium)', 'fetal_Capillaries', 'fetal_Neutrophil2', 'fetal_Capillary arterioles', 'fetal_LC', 'fetal_Megakaryocyte', 'fetal_MEMP - Megak', 'fetal_Mast cell (earliest)', 'fetal_Erythroid (fetal)', 'fetal_Myoblasts', 'fetal_Myofibroblasts', 'fetal_Pre-dermal condensate', 'fetal_HOXC5+ early fibroblast', 'fetal_PEAR1+ fibroblast', 'fetal_Pericytes',

In [19]:
del pdata2.obsm['X_pca_project']
del pdata2.obsm['X_umap']
pdata2

AnnData object with n_obs × n_vars = 300811 × 19372
    obs: 'sanger_id', 'chemistry', 'batch', 'gender', 'pcw', 'sorting', 'sample', 'chemistry_sorting', 'day', 'sample_id', 'version', 'week', 'org_annot30', 'dataset', 'joint_annotation_20220202', 'fig1b_annotation_v2', 'independent_annotation_refined_20220202', 'annot', 'org_annot0', 'pooled_annot'

In [20]:
#save data object to be converted from h5ad to rds
pdata2.write_h5ad('/nfs/team298/hg6/Fetal_skin/data/FS_organoid_Nichenet.h5ad')

... storing 'sanger_id' as categorical
... storing 'chemistry' as categorical
... storing 'batch' as categorical
... storing 'gender' as categorical
... storing 'sorting' as categorical
... storing 'sample' as categorical
... storing 'chemistry_sorting' as categorical
... storing 'day' as categorical
... storing 'sample_id' as categorical
... storing 'version' as categorical
... storing 'week' as categorical
... storing 'org_annot30' as categorical
... storing 'dataset' as categorical
... storing 'joint_annotation_20220202' as categorical
... storing 'fig1b_annotation_v2' as categorical
... storing 'independent_annotation_refined_20220202' as categorical
... storing 'annot' as categorical
... storing 'org_annot0' as categorical
... storing 'pooled_annot' as categorical
